In [1]:
import duckdb

# puzzle input goes here
input = """
"""

In [2]:
duckdb.sql("""
    WITH input AS (
        SELECT '%s' AS input
    ), t AS (
        SELECT
            UNNEST(CAST(REGEXP_EXTRACT_ALL(STRING_SPLIT(input, '\n')[1], '([0-9]+)', 1) AS INT[])) i_time,
            UNNEST(CAST(REGEXP_EXTRACT_ALL(STRING_SPLIT(input, '\n')[2], '([0-9]+)', 1) AS INT[])) i_dist
        FROM input
    ), run AS (
        SELECT
            i_time,
            i_dist,
            (
                SELECT COUNT(*)
                FROM UNNEST(t_hold) t_hold(v)
                WHERE (i_time - v) * v > i_dist
            ) n
        FROM (
            SELECT i_time, i_dist, GENERATE_SERIES(1, i_time) t_hold
            FROM t
        )
    )
    SELECT CAST(PRODUCT(n) AS INT) ans
    FROM run
""" % input)

┌─────────┐
│   ans   │
│  int32  │
├─────────┤
│ 1159152 │
└─────────┘

In [3]:
duckdb.sql("""
    WITH input AS (
        SELECT '%s' AS input
    ), t AS (
        SELECT
            UNNEST(CAST(REGEXP_EXTRACT_ALL(REPLACE(STRING_SPLIT(input, '\n')[1], ' ', ''), '([0-9]+)', 1) AS HUGEINT[])) i_time,
            UNNEST(CAST(REGEXP_EXTRACT_ALL(REPLACE(STRING_SPLIT(input, '\n')[2], ' ', ''), '([0-9]+)', 1) AS HUGEINT[])) i_dist
        FROM input
    ), min_max AS (
        SELECT
            -- (i_time - v) * v > i_dist
            -- v*v - v*i_time + i_dist > 0
            -- roots:
            -- x = (-b +- (b*b - 4*a*c)^(0.5)) / (2 * a)
            -- x = v, a = 1, b = -1*i_time, c = i_dist
            -- v = ( i_time +- ( (-1*i_time)*(-1*i_time) - 4*1*i_dist)^(0.5) ) / 2
            CAST(CEIL(( i_time - ( (-1*i_time)*(-1*i_time) - 4*1*i_dist)^(0.5) ) / 2) AS INT) v_min,
            CAST(FLOOR(( i_time + ( (-1*i_time)*(-1*i_time) - 4*1*i_dist)^(0.5) ) / 2) AS INT) v_max,
        FROM t
    )
    SELECT v_max - v_min + 1 ans
    FROM min_max
""" % input)

┌──────────┐
│   ans    │
│  int32   │
├──────────┤
│ 41513103 │
└──────────┘